In [130]:
from vqf.preprocessing import *
from itertools import product
import pandas as pd
from IPython.display import display
from sympy import simplify

In [131]:
p = 11
q = 13
m = p * q

_, _, _, clauses = create_clauses(m, apply_preprocessing=False, verbose=False)

for c in clauses:
    display(c)

p_0*q_0 - 1

p_0*q_1 + p_1*q_0 - 2*z_1_2 - 4*z_1_3 - 8*z_1_4 - 16*z_1_5 - 32*z_1_6 - 64*z_1_7 - 1

p_0*q_2 + p_1*q_1 + p_2*q_0 + z_1_2 - 2*z_2_3 - 4*z_2_4 - 8*z_2_5 - 16*z_2_6 - 32*z_2_7 - 1

p_0*q_3 + p_1*q_2 + p_2*q_1 + p_3*q_0 + z_1_3 + z_2_3 - 2*z_3_4 - 4*z_3_5 - 8*z_3_6 - 16*z_3_7 - 1

p_1*q_3 + p_2*q_2 + p_3*q_1 + p_4*q_0 + z_1_4 + z_2_4 + z_3_4 - 2*z_4_5 - 4*z_4_6 - 8*z_4_7

...

p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6*q_0 + z_1_6 + z_2_6 + z_3_6 + z_4_6 + z_5_6 - 2*z_6_7

p_4*q_3 + p_5*q_2 + p_6*q_1 + p_7*q_0 + z_1_7 + z_2_7 + z_3_7 + z_4_7 + z_5_7 + z_6_7 - 1

p_5*q_3 + p_6*q_2 + p_7*q_1

p_6*q_3 + p_7*q_2

p_7*q_3

In [132]:
_, _, _, simplified_clauses = create_clauses(m, apply_preprocessing=True, verbose=False)
for c in simplified_clauses:
    display(c)

0

0

0

p_3 - 2*q_1*q_2 + q_1 + q_2 + q_3 - 2*z_3_4 - 1

p_3*q_1 + p_4 - q_1*q_3 + q_3 + z_3_4 - 2*z_4_5 - 4*z_4_6

...

p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6 + z_4_6 + z_5_6 - 2*z_6_7

p_4*q_3 + p_5*q_2 + p_6*q_1 + p_7 + z_5_7 + z_6_7 - 1

p_5*q_3 + p_6*q_2 + p_7*q_1

p_6*q_3 + p_7*q_2

p_7*q_3

In [133]:
def ignore_z_terms(clause):
    # Create a dictionary to substitute z terms with 0
    subs_dict = {var: 0 for var in clause.free_symbols if str(var).startswith('z_')}

    # Substitute z terms with 0 in the expression
    final_clause = clause.subs(subs_dict)
    return final_clause

non_z_clauses = []
for c in simplified_clauses:
    final_clause = ignore_z_terms(c)
    non_z_clauses.append(final_clause)

In [134]:
for c in non_z_clauses:
    display(c)

0

0

0

p_3 - 2*q_1*q_2 + q_1 + q_2 + q_3 - 1

p_3*q_1 + p_4 - q_1*q_3 + q_3

...

p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6

p_4*q_3 + p_5*q_2 + p_6*q_1 + p_7 - 1

p_5*q_3 + p_6*q_2 + p_7*q_1

p_6*q_3 + p_7*q_2

p_7*q_3

In [135]:
def reduce_space(expr):
    variables = list(expr.free_symbols)
    values = product(range(2), repeat=len(variables))
    reduced_space = []
    for v in values:
        subs_dict = dict(zip(variables, v))
        subs_expr = expr.subs(subs_dict)
        if simplify(subs_expr) == 0:
            reduced_space.append(v)
    return variables, reduced_space

In [136]:
import itertools 
disjoint_clauses = []

var_sets = [c.free_symbols for c in non_z_clauses]

for i in range(len(var_sets)):
        for j in range(i + 1, len(var_sets)):
            if bool(var_sets[i]) and bool(var_sets[j]):
                if var_sets[i].isdisjoint(var_sets[j]):
                    disjoint_clauses.append(non_z_clauses[i])
                    disjoint_clauses.append(non_z_clauses[j])
            else: 
                 continue

for c in disjoint_clauses:
    display(c)          

In [137]:
frames = []
for c in disjoint_clauses:
    if c != 0:
        variables, reduced_space = reduce_space(c)
        df = pd.DataFrame(reduced_space, columns=variables)
        frames.append(df)

In [138]:
for df in frames:
    display(df)